##### THE ULTIMATE START OF USING THE MAPPERS TO CREATE AND QUERY TABLES


##### 1. Introduction

- importing the necessary modules to use in the course


In [1]:
from sqlalchemy.orm import Mapped, mapped_column,DeclarativeBase,relationship,Session
from sqlalchemy import create_engine,MetaData,ForeignKey,select,insert,func,text
import pandas as pd
import datetime
from typing import List

##### 2. creating a connection


In [2]:
engine = create_engine('sqlite:///test.db',echo=True)

# Each call to get_db() now creates a new session, which is safer and avoids locks.
def get_db():
    db = Session(engine)
    try:
        yield db
    finally:
        db.close()
        
print("connection success")

connection success


##### 3. Creating Models


In [3]:
# Define base class for SQLAlchemy ORM models
class Base(DeclarativeBase):
    pass

# Define User model
class User(Base):
    __tablename__ = "users"
    
    # Primary key
    id: Mapped[int] = mapped_column(primary_key=True)
    
    # User information
    username: Mapped[str] = mapped_column(nullable=False)
    email: Mapped[str] = mapped_column(unique=True, nullable=False)
    
    # Automatically set creation date to today
    created_at: Mapped[datetime.date] = mapped_column(default=datetime.date.today)
    
    # Relationship: One user has many orders
    orders: Mapped[List["Order"]] = relationship(back_populates="user")
    
    def __repr__(self) -> str:
        return f"User(id={self.id}, username={self.username}, email={self.email})"

# Define Order model
class Order(Base):
    __tablename__ = "orders"
    
    # Primary key
    order_id: Mapped[int] = mapped_column(primary_key=True)
    
    # Foreign key to users table
    user_id: Mapped[int] = mapped_column(ForeignKey("users.id"))
    
    # Order details
    item: Mapped[str] = mapped_column(nullable=False)
    price: Mapped[float] = mapped_column(nullable=False)
    quantity: Mapped[int]
    
    # Relationship: Each order belongs to one user
    user: Mapped["User"] = relationship(back_populates="orders")
    
    def __repr__(self) -> str:
        return (
            f"Order(order_id={self.order_id}, item={self.item}, "
            f"price={self.price}, quantity={self.quantity})"
        )

# Create tables in the database
Base.metadata.create_all(engine)
print("Tables created successfully.")


2025-10-16 19:47:25,444 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 19:47:25,455 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2025-10-16 19:47:25,461 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-16 19:47:25,469 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("orders")
2025-10-16 19:47:25,478 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-10-16 19:47:25,483 INFO sqlalchemy.engine.Engine COMMIT
Tables created successfully.


##### 4. Inserting the data

- a. Inserting the data using self defined values


In [5]:
#define the users
users = [
    User(username="Kilonzo", email="kilo@gmail.com"),
    User(username="Masila", email="kimeu@gmail.com"),
    User(username="Jane", email="jane@gmail.com"),
    User(username="James", email="james@gmail.com"),
    User(username="Mali", email="mali@gmail.com"),
    User(username="Juma", email="juma@gmail.com"),
]

for db in get_db():
    db.add_all(users)
    db.commit()
    
    # Refresh each user to get their assigned database ID
    for user in users:
        db.refresh(user)
    
    # Define realistic orders using actual user IDs
    orders = [
        Order(user_id=users[0].id, item="Brooms", price=30, quantity=3),
        Order(user_id=users[1].id, item="Gas Cylinder", price=2500, quantity=1),
        Order(user_id=users[0].id, item="Mop", price=34, quantity=3),
        Order(user_id=users[2].id, item="Mangoes", price=90, quantity=37),
        Order(user_id=users[1].id, item="Basins", price=76, quantity=9),
        Order(user_id=users[3].id, item="Brooms", price=76, quantity=33),
        Order(user_id=users[4].id, item="Laptop", price=65000, quantity=1),
        Order(user_id=users[5].id, item="Notebook", price=150, quantity=10),
        Order(user_id=users[2].id, item="Headphones", price=2000, quantity=2),
        Order(user_id=users[3].id, item="Towels", price=350, quantity=5),
        Order(user_id=users[4].id, item="Phone Charger", price=500, quantity=4),
        Order(user_id=users[5].id, item="School Bag", price=1200, quantity=2),
    ]
    
    db.add_all(orders)
    db.commit()

print("first batch of users and orders added successifully")

2025-10-16 19:48:23,996 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 19:48:24,002 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:24,007 INFO sqlalchemy.engine.Engine [cached since 50.13s ago (insertmanyvalues) 1/6 (ordered; batch not supported)] ('Kilonzo', 'kilo@gmail.com', '2025-10-16')
2025-10-16 19:48:24,011 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:24,014 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/6 (ordered; batch not supported)] ('Masila', 'kimeu@gmail.com', '2025-10-16')
2025-10-16 19:48:24,017 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:24,021 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/6 (ordered; batch not supported)] ('Jane', 'jane@gmail.com', '2025-10-16')
2025-10-16 19:48:24,025 INFO sqlalchemy.engine.Engine 

- b. Inserting using the faker


In [6]:
# User and Order models assumed already defined
from faker import Faker
import random

fake = Faker()
Faker.seed(0) # for reproducibility
# Create fake users
users = [
    User(
        username=fake.first_name(),
        email=fake.unique.email(),
    )
    for _ in range(10)  # generate 10 users
]

# Insert users and refresh to get their IDs
for db in get_db():
    db.add_all(users)
    db.commit()
    for user in users:
        db.refresh(user)

    # Generate random orders for these users
    items = ['Laptop', 'Phone', 'Book', 'Mug', 'Keyboard', 'T-Shirt', 'Pen', 'Shoes']
    
    orders = []
    for _ in range(20):  # create 20 orders
        orders.append(Order(
            user_id=random.choice(users).id,
            item=random.choice(items),
            price=round(random.uniform(10, 1000), 2),  # random price between 10 and 1000
            quantity=random.randint(1, 5)
        ))

    db.add_all(orders)
    db.commit()

print("Added more fake orders")
        

2025-10-16 19:48:39,160 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 19:48:39,162 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:39,165 INFO sqlalchemy.engine.Engine [cached since 65.29s ago (insertmanyvalues) 1/10 (ordered; batch not supported)] ('Megan', 'gwilliams@example.com', '2025-10-16')
2025-10-16 19:48:39,169 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:39,173 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/10 (ordered; batch not supported)] ('Richard', 'tammy59@example.org', '2025-10-16')
2025-10-16 19:48:39,176 INFO sqlalchemy.engine.Engine INSERT INTO users (username, email, created_at) VALUES (?, ?, ?) RETURNING id
2025-10-16 19:48:39,180 INFO sqlalchemy.engine.Engine [insertmanyvalues 3/10 (ordered; batch not supported)] ('William', 'donald19@example.com', '2025-10-16')
2025-10-16 19:48:39,182 INFO sql

- c. Inserting data using the insert keyword


In [8]:
stmt = (
    insert(Order).values([
        {"user_id": 2, "item": "air conditioner", "price": 1200, "quantity": 1},
        {"user_id": 4, "item": "washing machine", "price": 850, "quantity": 1},
        {"user_id": 5, "item": "electric scooter", "price": 1500, "quantity": 2},
        {"user_id": 3, "item": "notebook", "price": 150, "quantity": 5}
    ])
)

for db in get_db():
    db.execute(stmt)
    db.commit()

print("More data added using insert()")


2025-10-16 19:53:35,046 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 19:53:35,050 INFO sqlalchemy.engine.Engine INSERT INTO orders (user_id, item, price, quantity) VALUES (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?), (?, ?, ?, ?)
2025-10-16 19:53:35,052 INFO sqlalchemy.engine.Engine [no key 0.00211s] (2, 'air conditioner', 1200.0, 1, 4, 'washing machine', 850.0, 1, 5, 'electric scooter', 1500.0, 2, 3, 'notebook', 150.0, 5)
2025-10-16 19:53:35,056 INFO sqlalchemy.engine.Engine COMMIT
More data added using insert()


##### 5. Querying the data


- a. Get the data and its related orders


In [37]:
dfu =[]
for db in get_db():
    all_users = db.query(User).all()
    
    for user in all_users:
        serialized_orders =[
            {"item":order.item,
             "price":order.price,
             "quantity":order.quantity
            }
            for order in user.orders
        ]
        dfu.append({
            "username":user.username,
            "email":user.email,
            "created_at":user.created_at,
            "orders":serialized_orders
        })
  
df = pd.DataFrame(dfu)
df

2025-10-16 20:41:05,446 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:41:05,449 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.created_at AS users_created_at 
FROM users
2025-10-16 20:41:05,451 INFO sqlalchemy.engine.Engine [cached since 2793s ago] ()
2025-10-16 20:41:05,455 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders 
WHERE ? = orders.user_id
2025-10-16 20:41:05,458 INFO sqlalchemy.engine.Engine [cached since 1233s ago] (1,)
2025-10-16 20:41:05,461 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders 
WHERE ? = orders.user_id
2025-10-16 20:41:05,463 INFO sqlalchemy.eng

,username,email,created_at,orders
0,Kilonzo,kilo@gmail.com,2025-10-16,"[{'item': 'Brooms', 'price': 30.0, 'quantity':..."
1,Masila,kimeu@gmail.com,2025-10-16,"[{'item': 'Gas Cylinder', 'price': 2500.0, 'qu..."
2,Jane,jane@gmail.com,2025-10-16,"[{'item': 'Mangoes', 'price': 90.0, 'quantity'..."
3,James,james@gmail.com,2025-10-16,"[{'item': 'Brooms', 'price': 76.0, 'quantity':..."
4,Mali,mali@gmail.com,2025-10-16,"[{'item': 'Laptop', 'price': 65000.0, 'quantit..."
5,Juma,juma@gmail.com,2025-10-16,"[{'item': 'Notebook', 'price': 150.0, 'quantit..."
6,Megan,gwilliams@example.com,2025-10-16,"[{'item': 'Keyboard', 'price': 550.77, 'quanti..."
7,Richard,tammy59@example.org,2025-10-16,"[{'item': 'Laptop', 'price': 799.1, 'quantity'..."
8,William,donald19@example.com,2025-10-16,"[{'item': 'Mug', 'price': 13.29, 'quantity': 2..."
9,Vanessa,kyleblair@example.net,2025-10-16,"[{'item': 'Laptop', 'price': 490.69, 'quantity..."


- Querying the data (you can check the data more)


In [46]:
pd.DataFrame(df.loc[1]['orders'])

,item,price,quantity
0,Gas Cylinder,2500.0,1
1,Basins,76.0,9
2,air conditioner,1200.0,1


- b. To make a table for easy analysis (users join orders)


In [48]:
order_data = []

for db in get_db():
    all_users = db.query(User).all()

    for user in all_users:
        for order in user.orders:
            order_data.append({
                "username": user.username,
                "email": user.email,
                "item": order.item,
                "price": order.price,
                "quantity": order.quantity,
                "created_at": user.created_at
            })

orders_df = pd.DataFrame(order_data)
orders_df


2025-10-16 20:45:03,246 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:45:03,248 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.created_at AS users_created_at 
FROM users
2025-10-16 20:45:03,250 INFO sqlalchemy.engine.Engine [cached since 3031s ago] ()
2025-10-16 20:45:03,256 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders 
WHERE ? = orders.user_id
2025-10-16 20:45:03,259 INFO sqlalchemy.engine.Engine [cached since 1471s ago] (1,)
2025-10-16 20:45:03,263 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders 
WHERE ? = orders.user_id
2025-10-16 20:45:03,265 INFO sqlalchemy.eng

,username,email,item,price,quantity,created_at
0,Kilonzo,kilo@gmail.com,Brooms,30.00,3,2025-10-16
1,Kilonzo,kilo@gmail.com,Mop,34.00,3,2025-10-16
2,Masila,kimeu@gmail.com,Gas Cylinder,2500.00,1,2025-10-16
3,Masila,kimeu@gmail.com,Basins,76.00,9,2025-10-16
4,Masila,kimeu@gmail.com,air conditioner,1200.00,1,2025-10-16
5,Jane,jane@gmail.com,Mangoes,90.00,37,2025-10-16
6,Jane,jane@gmail.com,Headphones,2000.00,2,2025-10-16
7,Jane,jane@gmail.com,notebook,150.00,5,2025-10-16
8,James,james@gmail.com,Brooms,76.00,33,2025-10-16
9,James,james@gmail.com,Towels,350.00,5,2025-10-16


- c. using selectin load


In [50]:
from sqlalchemy.orm import selectinload

stmt = select(User).options(selectinload(User.orders))

for db in get_db():
    users = db.scalars(stmt).all()
    data =[]
    for user in users:
        for order in user.orders:
            data.append({
            "user_id": user.id,
            "name": user.username,
            "email": user.email,
            "order_id": order.order_id,
            "item": order.item,
            "quantity": order.quantity,
            "price": order.price,
            "created_at":user.created_at
        })
df = pd.DataFrame(data)    
df


2025-10-16 20:45:46,668 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:45:46,674 INFO sqlalchemy.engine.Engine SELECT users.id, users.username, users.email, users.created_at 
FROM users
2025-10-16 20:45:46,677 INFO sqlalchemy.engine.Engine [cached since 3364s ago] ()
2025-10-16 20:45:46,704 INFO sqlalchemy.engine.Engine SELECT orders.user_id AS orders_user_id, orders.order_id AS orders_order_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders 
WHERE orders.user_id IN (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
2025-10-16 20:45:46,710 INFO sqlalchemy.engine.Engine [cached since 3364s ago] (1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)
2025-10-16 20:45:46,725 INFO sqlalchemy.engine.Engine ROLLBACK


,user_id,name,email,order_id,item,quantity,price,created_at
0,1,Kilonzo,kilo@gmail.com,1,Brooms,3,30.00,2025-10-16
1,1,Kilonzo,kilo@gmail.com,3,Mop,3,34.00,2025-10-16
2,2,Masila,kimeu@gmail.com,2,Gas Cylinder,1,2500.00,2025-10-16
3,2,Masila,kimeu@gmail.com,5,Basins,9,76.00,2025-10-16
4,2,Masila,kimeu@gmail.com,33,air conditioner,1,1200.00,2025-10-16
5,3,Jane,jane@gmail.com,4,Mangoes,37,90.00,2025-10-16
6,3,Jane,jane@gmail.com,9,Headphones,2,2000.00,2025-10-16
7,3,Jane,jane@gmail.com,36,notebook,5,150.00,2025-10-16
8,4,James,james@gmail.com,6,Brooms,33,76.00,2025-10-16
9,4,James,james@gmail.com,10,Towels,5,350.00,2025-10-16


- d. Using the `select` to query and group data


In [51]:

stmt = (
    select(Order.item,func.sum(Order.price  * Order.quantity).label("revenue"))
    .group_by(Order.item)
)

for db in get_db():
    df = pd.DataFrame(db.execute(stmt))
df    

2025-10-16 20:50:27,250 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:50:27,254 INFO sqlalchemy.engine.Engine SELECT orders.item, sum(orders.price * orders.quantity) AS revenue 
FROM orders GROUP BY orders.item
2025-10-16 20:50:27,256 INFO sqlalchemy.engine.Engine [cached since 3287s ago] ()
2025-10-16 20:50:27,263 INFO sqlalchemy.engine.Engine ROLLBACK


,item,revenue
0,Basins,684.00
1,Book,8462.14
2,Brooms,2598.00
3,Gas Cylinder,2500.00
4,Headphones,4000.00
5,Keyboard,550.77
6,Laptop,71301.10
7,Mangoes,3330.00
8,Mop,102.00
9,Mug,4416.08


##### 6. updation and deletion


In [52]:
#update
for db in get_db():
    user = db.get(User,2)
    user.username = "kimeu Masila"
    user.email = "kimeudan05@gmail.com"
    db.commit()
    
# delete
for db in get_db():
    order = db.query(Order).first()
    db.delete(order)
    db.commit()
    
print("updated ad deleted successifully")

2025-10-16 20:50:53,931 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:50:53,940 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.username AS users_username, users.email AS users_email, users.created_at AS users_created_at 
FROM users 
WHERE users.id = ?
2025-10-16 20:50:53,942 INFO sqlalchemy.engine.Engine [generated in 0.00194s] (2,)


2025-10-16 20:50:53,982 INFO sqlalchemy.engine.Engine UPDATE users SET username=?, email=? WHERE users.id = ?
2025-10-16 20:50:53,987 INFO sqlalchemy.engine.Engine [generated in 0.00477s] ('kimeu Masila', 'kimeudan05@gmail.com', 2)
2025-10-16 20:50:54,001 INFO sqlalchemy.engine.Engine COMMIT
2025-10-16 20:50:54,017 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-10-16 20:50:54,024 INFO sqlalchemy.engine.Engine SELECT orders.order_id AS orders_order_id, orders.user_id AS orders_user_id, orders.item AS orders_item, orders.price AS orders_price, orders.quantity AS orders_quantity 
FROM orders
 LIMIT ? OFFSET ?
2025-10-16 20:50:54,027 INFO sqlalchemy.engine.Engine [generated in 0.00364s] (1, 0)
2025-10-16 20:50:54,039 INFO sqlalchemy.engine.Engine DELETE FROM orders WHERE orders.order_id = ?
2025-10-16 20:50:54,044 INFO sqlalchemy.engine.Engine [generated in 0.00501s] (1,)
2025-10-16 20:50:54,051 INFO sqlalchemy.engine.Engine COMMIT
updated ad deleted successifully
